In [0]:
import pandas as pd
import numpy as np
import xgboost as xgb
import nltk
from tqdm import tqdm
from sklearn.svm import SVC
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
from nltk import word_tokenize
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')


Using TensorFlow backend.


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
def multiclass_logloss(actual, predicted, eps=1e-15):
    """Multi class version of Logarithmic Loss metric.
    :param actual: Array containing the actual target classes
    :param predicted: Matrix with class predictions, one probability per class
    """
    # Convert 'actual' to a binary array if it's not already:
    if len(actual.shape) == 1:
        actual2 = np.zeros((actual.shape[0], predicted.shape[1]))
        for i, val in enumerate(actual):
            actual2[i, val] = 1
        actual = actual2

    clip = np.clip(predicted, eps, 1 - eps)
    rows = actual.shape[0]
    vsota = np.sum(actual * np.log(clip))
    return -1.0 / rows * vsota

In [0]:
train= pd.read_csv("/content/drive/My Drive/340 project/train.tsv", sep="\t")
test = pd.read_csv("/content/drive/My Drive/340 project/test.tsv", sep="\t")
sam=pd.read_csv("/content/drive/My Drive/340 project/sampleSubmission.csv")

train.head()

,PhraseId,SentenceId,Phrase,Sentiment
0,1,1,A series of escapades demonstrating the adage ...,1
1,2,1,A series of escapades demonstrating the adage ...,2
2,3,1,A series,2
3,4,1,A,2
4,5,1,series,2


In [0]:
print(train.shape)
print(test.shape)

(156060, 4)
(66292, 3)


In [0]:
xtrain, xvalid, ytrain, yvalid = train_test_split(train["Phrase"], train["Sentiment"], 
                                                  stratify=train["Sentiment"], 
                                                  random_state=42, 
                                                  test_size=0.1, shuffle=True)

TF+logistic


In [0]:
tfv = TfidfVectorizer(min_df=5,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1,
            stop_words = 'english')
tfv.fit(list(xtrain) + list(xvalid))
xtrain_tfv =  tfv.transform(xtrain) 
xvalid_tfv = tfv.transform(xvalid)

In [0]:
print(xtrain_tfv.shape,xvalid_tfv.shape)

(140454, 88997) (15606, 88997)


In [0]:
clf = LogisticRegression(C=1.0)
clf.fit(xtrain_tfv, ytrain)
predictions = clf.predict(xvalid_tfv)




/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [0]:
sum(predictions==yvalid)/len(predictions)

0.6402024862232475

In [0]:
xtest_tfv = tfv.transform(test["Phrase"])

In [0]:
predictions = clf.predict(xtest_tfv)

In [0]:
sam["Sentiment"]=predictions

In [0]:
sam.to_csv("/content/drive/My Drive/340 project/1.csv",index=False)

CountVectorizer + logistics


In [0]:
ctv = CountVectorizer(analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), stop_words = 'english')

ctv.fit(list(xtrain) + list(xvalid))
xtrain_ctv =  ctv.transform(xtrain) 
xvalid_ctv = ctv.transform(xvalid)

In [0]:
clf = LogisticRegression(C=1.0)
clf.fit(xtrain_ctv, ytrain)
predictions = clf.predict(xvalid_ctv)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [0]:
predictions

array([1, 2, 2, ..., 0, 1, 2])

In [0]:
sum(predictions==yvalid)/len(predictions)

0.6630142252979623

In [0]:
xtest_tfv = ctv.transform(test["Phrase"])

In [0]:
predictions = clf.predict(xtest_tfv)

In [0]:
sam["Sentiment"]=predictions

In [0]:
sam.to_csv("/content/drive/My Drive/340 project/2.csv",index=False)


TF+MultinomialNB

In [0]:
clf = MultinomialNB()
clf.fit(xtrain_tfv, ytrain)
predictions = clf.predict(xvalid_tfv)

In [0]:
sum(predictions==yvalid)/len(predictions)

0.6325131359733436

In [0]:
xtest_tfv = tfv.transform(test["Phrase"])

In [0]:
predictions = clf.predict(xtest_tfv)

In [0]:
sam["Sentiment"]=predictions

In [0]:
sam.to_csv("/content/drive/My Drive/340 project/3.csv",index=False)

svm+tf


In [0]:

svd = decomposition.TruncatedSVD(n_components=120)
svd.fit(xtrain_tfv)
xtrain_svd = svd.transform(xtrain_tfv)
xvalid_svd = svd.transform(xvalid_tfv)


scl = preprocessing.StandardScaler()
scl.fit(xtrain_svd)
xtrain_svd_scl = scl.transform(xtrain_svd)
xvalid_svd_scl = scl.transform(xvalid_svd)

In [0]:
clf = SVC(C=1.0, probability=False) # since we need probabilities
clf.fit(xtrain_svd_scl, ytrain)
predictions = clf.predict(xvalid_svd_scl)


In [0]:
np.array(ytrain)

array([2, 2, 2, ..., 2, 1, 2])

xgb


In [0]:
dtrain = xgb.DMatrix(xtrain_tfv, label=np.array(ytrain) )
dtest=xgb.DMatrix(xvalid_tfv, label=np.array(yvalid))
evallist = [(dtest, 'eval'), (dtrain, 'train')]

In [0]:
param  ={'objective': 'multi:softmax','num_class':5}
param['eval_metric'] = 'mlogloss'
param['gpu_id'] = 0
param['tree_method'] = 'gpu_hist'
num_round=100000

In [0]:
bst = xgb.train(params=param,dtrain=dtrain,num_boost_round=num_round,evals=evallist,early_stopping_rounds=10)

In [0]:
predictions = xgb.predict(dtest)

NameError: ignored

In [0]:
clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)
clf.fit(xtrain_tfv.tocsc(), ytrain)
predictions = clf.predict(xvalid_tfv.tocsc())

In [0]:
sum(predictions==yvalid)/len(predictions)

0.567602204280405